In [1]:
from keras.applications.resnet50     import ResNet50
model = ResNet50(include_top=False, weights='imagenet', \
                    input_tensor=None, input_shape=None, pooling=None, classes=1000)

Using TensorFlow backend.
W0825 20:58:21.534365 140328432133952 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0825 20:58:21.955462 140328432133952 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 20:58:22.096666 140328432133952 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0825 20:58:22.287024 140328432133952 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/k

94658560/94653016 [==============================] - 5s 0us/step


In [3]:

import time

import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, AveragePooling2D, GlobalAveragePooling2D
from keras import backend as K

In [4]:
n_classes = 10
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

n_training = X_train.shape[0]
n_testing = X_test.shape[0]

y_train = y_train.flatten()
y_test  = y_test.flatten()

print( X_train.shape, y_train.shape,  X_test.shape, y_test.shape )

from matplotlib import pyplot as plt
plt.imshow( X_train[0]  )
plt.show()

170500096/170498071 [==============================] - 8s 0us/step
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


<Figure size 640x480 with 1 Axes>

In [5]:
from keras.models import Model

In [6]:
input_shape = {
    'incv3'   : (299,299,3),
    'resnet50': (224,224,3),
    'vgg16'   : (224,224,3),
    'vgg19'   : (224,224,3)
}['resnet50']

In [7]:
def create_model_resnet50():
    tf_input = Input(shape=input_shape)
    base_model = ResNet50(input_tensor=tf_input, include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return base_model, model

In [8]:
batch_of_images_placeholder = tf.placeholder("uint8", (None, 32, 32, 3))

In [9]:
tf_resize_op = tf.image.resize_images(batch_of_images_placeholder, (input_shape[:2]), method=0)

In [10]:
from keras.applications.resnet50     import preprocess_input as resnet50_preprocess_input

In [11]:
preprocess_input = {
  
    'resnet50': resnet50_preprocess_input,

}['resnet50']

In [12]:
def data_generator(sess,data,labels):
    def generator():
        start = 0
        end = start + 32
        n = data.shape[0]
        while True:
            batch_of_images_resized = sess.run(tf_resize_op, {batch_of_images_placeholder: data[start:end]})
            batch_of_images__preprocessed = preprocess_input(batch_of_images_resized)
            batch_of_labels = labels[start:end]
            
            start += 32
            end   += 32
            if start >= n:
                start = 0
                end = 32
            yield (batch_of_images__preprocessed, batch_of_labels)
    return generator

In [13]:
sess = tf.InteractiveSession()

In [14]:
K.set_session(sess)
K.set_learning_phase(1)

In [15]:
base_model, model = create_model_resnet50()

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [16]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_2
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_50
5 pool1_pad
6 max_pooling2d_2
7 res2a_branch2a
8 bn2a_branch2a
9 activation_51
10 res2a_branch2b
11 bn2a_branch2b
12 activation_52
13 res2a_branch2c
14 res2a_branch1
15 bn2a_branch2c
16 bn2a_branch1
17 add_17
18 activation_53
19 res2b_branch2a
20 bn2b_branch2a
21 activation_54
22 res2b_branch2b
23 bn2b_branch2b
24 activation_55
25 res2b_branch2c
26 bn2b_branch2c
27 add_18
28 activation_56
29 res2c_branch2a
30 bn2c_branch2a
31 activation_57
32 res2c_branch2b
33 bn2c_branch2b
34 activation_58
35 res2c_branch2c
36 bn2c_branch2c
37 add_19
38 activation_59
39 res3a_branch2a
40 bn3a_branch2a
41 activation_60
42 res3a_branch2b
43 bn3a_branch2b
44 activation_61
45 res3a_branch2c
46 res3a_branch1
47 bn3a_branch2c
48 bn3a_branch1
49 add_20
50 activation_62
51 res3b_branch2a
52 bn3b_branch2a
53 activation_63
54 res3b_branch2b
55 bn3b_branch2b
56 activation_64
57 res3b_branch2c
58 bn3b_branch2c
59 add_21
60 activation_65
61 res3c_branch2a
62

In [17]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_2
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_50
5 pool1_pad
6 max_pooling2d_2
7 res2a_branch2a
8 bn2a_branch2a
9 activation_51
10 res2a_branch2b
11 bn2a_branch2b
12 activation_52
13 res2a_branch2c
14 res2a_branch1
15 bn2a_branch2c
16 bn2a_branch1
17 add_17
18 activation_53
19 res2b_branch2a
20 bn2b_branch2a
21 activation_54
22 res2b_branch2b
23 bn2b_branch2b
24 activation_55
25 res2b_branch2c
26 bn2b_branch2c
27 add_18
28 activation_56
29 res2c_branch2a
30 bn2c_branch2a
31 activation_57
32 res2c_branch2b
33 bn2c_branch2b
34 activation_58
35 res2c_branch2c
36 bn2c_branch2c
37 add_19
38 activation_59
39 res3a_branch2a
40 bn3a_branch2a
41 activation_60
42 res3a_branch2b
43 bn3a_branch2b
44 activation_61
45 res3a_branch2c
46 res3a_branch1
47 bn3a_branch2c
48 bn3a_branch1
49 add_20
50 activation_62
51 res3b_branch2a
52 bn3b_branch2a
53 activation_63
54 res3b_branch2b
55 bn3b_branch2b
56 activation_64
57 res3b_branch2c
58 bn3b_branch2c
59 add_21
60 activation_65
61 res3c_branch2a
62

In [26]:
for layer in base_model.layers[:10]:
    layer.trainable = True
for layer in base_model.layers[10:]:
    layer.trainable = False

from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy')

In [27]:
y_train_one_hot = tf.one_hot( y_train, n_classes ).eval()

In [28]:
data_train_gen = data_generator(sess, X_train, y_train_one_hot )

In [29]:
model.fit_generator(data_train_gen(), n_training/32, verbose=1, epochs=20)

Epoch 1/20
1563/1562 [==============================] - 832s 532ms/step - loss: 0.1777
Epoch 2/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.1387
Epoch 3/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.1202
Epoch 4/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.1081
Epoch 5/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.1045
Epoch 6/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.0980
Epoch 7/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.0856
Epoch 8/20
1563/1562 [==============================] - 832s 532ms/step - loss: 0.0848
Epoch 9/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.0787
Epoch 10/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.0697
Epoch 11/20
1563/1562 [==============================] - 831s 532ms/step - loss: 0.0674
Epoch 12/20
1563/1562 [==================

In [30]:
images_resized = sess.run(tf_resize_op, {batch_of_images_placeholder: X_test})
images = preprocess_input(images_resized)

In [31]:
result = model.predict(images, verbose=1)

10000/10000 [==============================] - 78s 8ms/step


In [32]:
y_pred = [ np.argmax( result[i] ) for i in range(n_testing) ]

In [33]:
np.sum( y_pred == y_test ) / n_testing

0.8777